<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/Ejercicio_1_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programacion Concurrente - TP2 - Parte 1

Utilizamos 2 prácticas con OpenMP desarrolladas en lenguaje C/C++ nativo en plataforma colab. 


---
## 1.1. Ejercicio 1 - Hola Mundo estilo Programacion Concurrente


### 1.1.1. Preguntas ejercicio 1:

a) Identifique los 3 componentes de openMP en el ejercicio 1.

b) Realice pruebas modificando la cantidad de hilos (1, 3, 7). Que conclusiones pudo determinar?.

c) ¿Cuál es la diferencia las formas de asignación *static* y *dynamic* en la cláusula *schedule*?, ¿Qué sucede si utiliza *dynamic* en el código?

d) En el for: ¿Que sucede con el valor de j, sí se parametriza como lastprivate?


### Respuestas ejercicio 1:

a) 
Directivas al compilador: parallel, single, master, for.

Funciones de biblioteca: omp_get_thread_num(), omp_get_num_threads(), omp_get_num_procs() .

Variables de entorno: OMP_NUM_THREADS.

b) 
No importa con cuantos hilos ejecute el proceso, el número de procesadores de colab siempre me devuelve 2. En cada una de las ejecuciones que se intentaron, sin importar el número especificado de hilos (con la forma de asignación dynamic) es el hilo que ejecuta primero el for el que toma la mayor cantidad de registros. Al ejecutar para 1, 2 y 3 hilos, apenas 4 registros son tomados por un hilo diferente al que comenzó la ejecución. Es agregando más hilos donde las posibilidades de tener a más hilos ejecutando tareas en el for es mayor. 

c) 
En ambos tipos de asignación, se reparten las ejecuciones entre los threads. 
Si se utiliza la asignación static, las ejecuciones de cada thread se reparten antes de entrar al bucle. El valor del chunk es el que define cuantos valores se ejecutan por cada thread. El for llega hasta 20 iteraciones, y el chunk identifica 5 para cada thread. Si se usan 4 threads, cada uno ejecutará 5 veces. Si se definen menos de 4, cada hilo ejecutará como mínimo 5 veces, y luego se repartirán de 5 en 5 las ejecuciones restantes. (Si tengo 3 hilos, los hilos 0, 1 y 2 ejecutarán los primeros 15; y luego el hilo 0 ejecutará los restantes 5). Si utilizo más de 4 hilos, serán los primeros 4 en ser creados los que ejecutarán los valores. Dado a que está definido que como mínimo deben tomar 5, habrá hilos que no podrán ejecutar una iteración del for.
Si se utiliza la asignación dynamic, a los threads se les puede definir con el chunk cuantos datos ejecutar, pero una vez que hayan tomado los que les correspondían, pueden tomar incluso más. Por ejemplo, en una ejecución con 5 hilos, el hilo 2 tomó diez valores, el hilo 3 tomó cinco valores y el hilo 4 tomó otros cinco. La repartición no es de forma equitativa antes de empezar el proceso, dado a que los hilos 0 y 1 no ejecutaron ningún valor. 

d) 
Si se parametriza j como lastprivate, cada uno de los hilos tendrá una copia de la variable. Y la variable j global quedará actualizada con el valor que le dio el último hilo en ejecución. En caso de que el último hilo le asigne 5, ese será su valor al finalizar la ejecución. 

### 1.1.2. Código Lenguaje C 

In [ ]:
%%writefile code.cpp

// Hola Mundo desde OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <omp.h>    // Cabecera OpenMP   
  
int main(int argc, char* argv[]) 
{
  int i,j=0;
  std::cout<<"Inicio..."<<std::endl;

  //-----------------------------------------------------------------------------------------------
  // Region paralela
  #pragma omp parallel
  { 
    std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< ", de "<<  omp_get_num_threads() <<", procesadores "<< omp_get_num_procs()<< std::endl;  

    #pragma omp single
    {
      std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< " uno de muchos."<< std::endl;
    }
    #pragma omp master
    {
      std::cout<<"Hola mundo!!!... soy el hilo maestro: " << omp_get_thread_num()<< std::endl;
    }

    #pragma omp for schedule(dynamic, 5) lastprivate(j)
    for(i=0;i<20;i++)
    {
      j++;
      std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< ", itero para i ."<<i<<", actualizo j "<<j<< std::endl;
    }
  }
  // Region paralela
  //-----------------------------------------------------------------------------------------------

  std::cout<<"Fin..., con j="<<j<<std::endl;
}

Overwriting code.cpp


### 1.1.3 Compilación de Hola Mundo. 

In [ ]:
!g++ -o hello -fopenmp code.cpp

### 1.1.4. Ejecución Hola Mundo en OpenMP.

In [ ]:
%env OMP_NUM_THREADS=5
!./hello

env: OMP_NUM_THREADS=5
Inicio...
Hola mundo!!!... soy el hilo Hola mundo!!!... soy el hilo 20, de , de 55, procesadores , procesadores 22
Hola mundo!!!... soy el hilo 2 uno de muchos.

Hola mundo!!!... soy el hilo 4, de 5, procesadores 2
Hola mundo!!!... soy el hilo 1, de 5, procesadores 2
Hola mundo!!!... soy el hilo 3, de 5, procesadores 2
Hola mundo!!!... soy el hilo 3, itero para i .0, actualizo j 1
Hola mundo!!!... soy el hilo 3, itero para i .1, actualizo j 2
Hola mundo!!!... soy el hilo 3, itero para i .2, actualizo j 3
Hola mundo!!!... soy el hilo 3, itero para i .3, actualizo j 4
Hola mundo!!!... soy el hilo 3, itero para i .4, actualizo j 5
Hola mundo!!!... soy el hilo 3, itero para i .5, actualizo j 6
Hola mundo!!!... soy el hilo 3, itero para i .6, actualizo j 7
Hola mundo!!!... soy el hilo 3, itero para i .7, actualizo j 8
Hola mundo!!!... soy el hilo 3, itero para i .8, actualizo j 9
Hola mundo!!!... soy el hilo 3, itero para i .9, actualizo j 10
Hola mundo!!!... soy el h

---
## 1.2. Ejercicio 2 - Axpy

La función Axpy es parte de las bibliotecas BLAS [3]. Ella se encarga de realizar la suma de vectores, con la siguiente ecuación:

<center>$Y = \alpha X + Y$</center>

En donde:
> $\alpha$: Es un escala.

> $X$: Es el vector origen.

> $Y$: Es el vector destino.


### 1.2.1 Preguntas Ejercicio 2
a) Ejecute el ejercicio 1.2.2:

    - Con la variable cantidad_N para: 50, 500, 5.000.
    - Con los valores hilos_ 2, 4, 6.

b) Repita la prueba (a), con ciclos 1, 10, 30. ¿Cuál fue la diferencia?

c) ¿Por qué el SpeedUp no mejora a medida que aumentan la cantidad de hilos?

d) ¿Qué sucede con la eficiencia a medida que aumentan la cantidad_N?¿Porqué no llega al valor ideal?.

### Respuestas Ejercicio 2

a)
En la ejecución con valor de N 5000 y 2 hilos, el tiempo de la función por OMP fue más rápido que el tiempo por secuencial. En el resto de los casos, el tiempo de ejecución secuencial siempre fue más rápido. Si se utilizan 2 hilos, en los casos donde N vale 50 y 500, pese a ser más lento, la diferencia es de 0.07 milisegundos. Para cuando se ejecuta con 4 hilos o 6 hilos, la diferencia puede variar desde 1 milisegundo (caso N 500 y 6 threads) a 5 milisegundos (caso N 5000 y 6 threads).

b)
CON 30 CICLOS:
*   N 50: El valor del overhead al utilizar 4 hilos se redujo en 11 unidades. El valor del overhead al utilizar 6 hilos aumentó en 2 unidades. El valor del overhead al utilizar 2 hilos fue casi el mismo.
*   N 500: El valor del overhead al utilizar 2 hilos aumentó un total de 34 unidades. El overhead al utilizar 4 hilos aumentó apenas 1 unidad. Mientras que el del 6 se duplicó comparado a su valor en 10 ciclos.
*   N 5000: si se utilizan 4 hilos, el valor del overhead se reduce en 5 unidades. Al usarse 6 hilos, aumenta en 5 unidades. Si se utilizan 2, el overhead aumenta en 30 unidades. 

CON 100 CICLOS:
*   N 50: el valor del overhead al utilizar cualquier cantidad de hilos incrementa en gran medida a comparación con 10 ciclos. El overhead más alto cuando se usaron 10 ciclos fue de 13, mientras que el más alto fue de 23.
*   N 500: El overhead se inrementa demasiado al utilizar 100 ciclos. Al ejecutar con 10, el overhead más alto llega a 2.88, mientras que el más bajo al ejecutar con 100, es de 6.21. 
*   N 5000: al utilizar 100 ciclos, el overhead para todos los casos se incrementa. Siendo el incremento más pequeño al ejecutar con 6 hilos (aumenta 7 unidades) mientras que con 2 hilos aumenta 27.

c)
Para que el speed up pueda mejorar, se tiene que lograr que el tiempo que tarda en ejecutar la función Axpy en paralelo sea menor al tiempo que tarda en ejecutar de forma secuencial. No importa cuantos hilos se dediquen a ejecutar dicha función de forma paralela, el tiempo que tarda en procesar de manera secuencial es siempre más rápido, por lo que el speed up no va a mejorar. Incluso hay casos donde aumentar la cantidad de hilos hace que el tiempo demore todavía más. En las ejecuciones que se hicieron, usar 4 hilos demora más que usar 2, y usar 8 demora más que usar 4. Por esto mismo, el speed up únicamente va a disminuir. 

d)
Al ir incrementando el valor de N, el tiempo secuencial que tarda en ejecutar la función no se ve muy afectado (con N 50 y 2 hilos tarda 0.01 ms mientras que con N 5000 tarda 0.71 ms). En cambio, el tiempo que demora en ejecutar de forma paralela, con N 50 era de 2 ms mientras que al incrementarlo a N 5000 me llegó a dar una demora de 120 ms. El tiempo que tarda al aumentarse el valor de N se incrementa, pero es uno mucho mayor cuando la función ejecuta de forma paralela. Esto no hace más que disminuir el speed up. La eficiencia depende del speed up, dado a que en colab, el número de procesadores que tengo siempre será de 2, para obtener mejor eficiencia, es obligatorio mejorar el speed up. Algo que no ocurre al incrementar el valor de N.  

### 1.2.2 Código Lenguaje C 

In [ ]:
%%writefile code_axpy.cpp
// Axpy con OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

using namespace std;

// ----------------------------------------------------------------------------

void axpy( double alfa, vector<double> &X, vector<double> &Y )
{
  int i;

  #pragma omp parallel for
  for(i=0;i<Y.size();i++)
  {
    Y[i] = alfa * X[i] + Y[i];
  }
}

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  float alfa     = atof( argv[1] );
  int cantidad_N = atoi( argv[2] );
  int ciclos     = atoi( argv[3] );

  // --------------------------------------------
  // Defino la memoria de los vectores.

  vector<double> X( cantidad_N );
  vector<double> Y( cantidad_N );

  for (int i=0;i<X.size();i++)
  {
    X[i] = (rand()/(double)RAND_MAX)*0.73;
    Y[i] = (rand()/(double)RAND_MAX)*0.71;
  }

  // --------------------------------------------
  // Realizo la función Axpy con OpenMP.

  TIEMPO_INI( HTH_AXPY_OMP )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Realizo la función Axpy en forma secuencial.

  omp_set_num_threads(1);

  TIEMPO_INI( HTH_AXPY_SEC )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_SEC )

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / omp_get_num_procs();
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

Writing code_axpy.cpp


### 1.2.3. Compilación de código C Axpy. 

In [ ]:
!g++ -o axpy -fopenmp code_axpy.cpp

### 1.2.4. Ejecución Axpy.

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

Cantidad_N = 5000#@param {type: "number"}
Alfa       = 1.0#@param {type: "number"}
Ciclos     = 10#@param {type: "slider", min: 10, max: 200}
Hilos      = 2#@param {type: "slider", min: 1, max: 10}

# --------------------------------------------

%env OMP_NUM_THREADS=$Hilos
!./axpy $Alfa $Cantidad_N $Ciclos

env: OMP_NUM_THREADS=2
Valores Reales  :
Tiempo TOTAL     : 121.442 [ms]
Tiempo axpy Sec  : 0.714064 [ms]
Tiempo axpy Omp  : 120.439 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0.714064 / 120.439 = 0.00592884
Eficiencia       : SpeedUp/nro procesadores            : 0.00592884 / 2 = 0.00296442
Coste Sec        : nro procesadores*Tiempo             : 1 * 0.714064 = 0.714064
Coste Omp        : nro procesadores*Tiempo             : 2 * 120.439 = 240.878
Funcion Overhead : Coste Omp - tiempo Secuencial       : 240.878 - 0.714064 = 240.164

Valores Ideal: 
Tiempo axpy Sec  : 0.714064 [ms]
Tiempo axpy Omp  : 0.357032 [ms]
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0.714064 / 0.357032 = 2
Eficiencia       : SpeedUp/nro procesadores            : 2 / 2 = 1
Coste Sec        : nro procesadores*Tiempo             : 1 * 0.714064 = 0.714064
Coste Omp        : nro procesadores*Tiempo             : 2 * 0.357032 = 0.714064
Funcion Overhead : Coste Omp - tiempo Secuencial

---
# Medidas de prestaciones en algoritmos paralelos
Las técnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como medida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [6,7]. 


## SpeedUp
Referencia a la ganancia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = \frac{Tiempo\_Secuencial}{Tiempo\_Paralelo} $</center>



##Eficiencia
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganancia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = \frac{SpeedUp}{Nro\_Procesadores} $</center>


## Coste
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resolución del problema.

<center>$ coste = Nro\_Procesadores \times Tiempo\_Algoritmo$</center>


## Función Overhead
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.

<center>$ Overhead = {Coste}-{Tiempo\_Secuencial} $</center>

---

In [ ]:
!cat /proc/cpuinfo

---
# Bibliografía
[1] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[3] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[4] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[5] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://progc-unlam.com.ar/material-clase/OpenMP-MPI/Introducci%C3%B3n%20a%20la%20Computaci%C3%B3n%20Paralela.pdf)